In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier
import joblib

In [2]:
# variables globales
racine = '../s1-sd-projet/CK+'


pd.set_option('display.min_rows', 10)
pd.set_option('display.max_rows', 10)

In [3]:
# récupération du csv emotion ET des csv correspondant
metadata = pd.read_csv(racine + "/emotion.csv", sep=';')

### Préparation des données, on utilise les connaissances obtenues lors du projet pour préparer les données correctement.

In [7]:
# je crée des noms de colonnes pour les coordonnées des points
names = ['session'] + [str(coord) + str(i) for i in range(1, 69) for coord in ['x', 'y']]

# un dataframe vide
df = pd.DataFrame()

# pour chaque ligne dans le csv de metadata, je récupère le csv associé et l'emotion étudiée
# ajout de 'file' pour cibler la session spécifique
for subject, session_id, emotion in metadata[["subject", "file", "emotion"]].values:
    session_data = pd.read_csv(racine + f"./{subject}/omlands.csv", sep=';', names=names, usecols=range(137))
    
    # FILTRAGE : on ne garde que la session correspondant au fichier de metadata
    # file correspond à la première colonne 'session' du csv omlands
    session_data = session_data[session_data['session'] == int(session_id)]
    
    # insertion des metadata du sujet et de l'emotion
    session_data.insert(0, 'subject', subject)
    # NB : j'insère la target à la fin
    session_data['emotion'] = emotion

    # on concatène à la suite pour faire grossir le jeu de données
    df = pd.concat([df, session_data], ignore_index=True)

# On retire des lignes doubles ou incomplètes
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

# création de l'apex value pour gérer l'expression neutre
df.insert(2, 'apex_value', df.groupby(['subject', 'session']).cumcount())
# si on est au début du mouvement, alors c'est qu'on part d'une expression neutre, on choisi 0 pour l'émotion
df.loc[df['apex_value'] == 0, 'emotion'] = 0
# on garde uniquement les coordonnées et l'émotion
df = df.loc[:, "x1":]
# on retire à nouveau les lignes doubles ou incomplètes
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

### Entraînement du modèle MLPClassifier

In [ ]:
X = df.iloc[:, :-1].values
y = df['emotion'].values

# Utilisation du module pipline pour standardiser
pipeline_mlp = Pipeline([
    ('scaler', StandardScaler()),
    ('mlp', MLPClassifier(random_state=42))
])

# Hyperparametres finetunés pour MLPClassifier
param_grid_fine_3c = {
    'mlp__hidden_layer_sizes': [
        (100, 50)
    ],
    'mlp__activation': ['tanh'],  
    'mlp__alpha': [1e-6],  
    'mlp__learning_rate_init': [0.0001],  
    'mlp__max_iter': [200],  
}

grid_search = GridSearchCV(pipeline_mlp, param_grid_fine_3c, cv=StratifiedKFold(n_splits=5), scoring='accuracy', n_jobs=-1)
grid_search.fit(X, y)

print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)


Best parameters: {'mlp__activation': 'tanh', 'mlp__alpha': 1e-06, 'mlp__hidden_layer_sizes': (100, 50), 'mlp__learning_rate_init': 0.0001, 'mlp__max_iter': 200}
Best cross-validation score: 0.5783733085211478


c:\Users\Alex\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [6]:
#  export du model pour l'utiliser dans l'application avec import cv2 et dlib
joblib.dump(grid_search.best_estimator_, 'emotion_mlp_model.joblib')

['emotion_mlp_model.joblib']